In [1]:
# Change the directory path
%cd ../featureSelection/

C:\Users\mikec\Codes\DAADRISE_AbusiveLangProject\featureSelection


In [2]:
import pandas as pd
import h2o
h2o.init()
from h2o.automl import H2OAutoML
from sklearn import metrics

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from C:\Users\mikec\AppData\Local\Programs\Python\Python37\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\mikec\AppData\Local\Temp\tmpl2rn4een
  JVM stdout: C:\Users\mikec\AppData\Local\Temp\tmpl2rn4een\h2o_mikec_started_from_python.out
  JVM stderr: C:\Users\mikec\AppData\Local\Temp\tmpl2rn4een\h2o_mikec_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,29 days
H2O cluster name:,H2O_from_python_mikec_7z5ncz
H2O cluster total nodes:,1
H2O cluster free memory:,1.752 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [3]:
X_train = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\ImportanceFeatures_500.csv'))
y_train = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\TrainLabels.csv'))
X_test = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\ImportanceFeaturesTest_500.csv'))
y_test = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\TestLabels.csv')) 


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# preparing the train and test data sets
# now convert tweet vecs and labels to a pandas dataframe and back to h2o dataframe
# data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
train = X_train.cbind(y_train)
test = X_test.cbind(y_test)


In [5]:
# more on data prep
x = train.columns         # x: A list/vector of predictor column names or indexes. 
                          # This argument only needs to be specified if the user wants to exclude columns from the 
                          # set of predictors. If all columns (other than the response) should be used in prediction, 
                          # then this does not need to be set.

y = "labels"              # This argument is the name (or index) of the response column
x.remove(y)

# need to set train and test
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [6]:
# now the AUTO-ML piece comes in
aml = H2OAutoML(max_models=10) #max_models=10 or 20?, max_runtime_secs=3600
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [7]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_1_AutoML_20190718_142918,0.666351,0.678094,0.457068,0.208911
DRF_1_AutoML_20190718_142918,0.666381,0.679683,0.458117,0.209871
GBM_1_AutoML_20190718_142918,0.666578,0.66671,0.459328,0.210982
GBM_3_AutoML_20190718_142918,0.66661,0.668056,0.459466,0.211109
GBM_grid_1_AutoML_20190718_142918_model_1,0.666667,0.90129,0.590793,0.349036
GBM_5_AutoML_20190718_142918,0.666667,0.669862,0.459751,0.211371
StackedEnsemble_AllModels_AutoML_20190718_142918,0.666667,0.663647,0.455118,0.207133
GLM_grid_1_AutoML_20190718_142918_model_1,0.666667,0.662468,0.454826,0.206867
StackedEnsemble_BestOfFamily_AutoML_20190718_142918,0.666667,0.662707,0.454882,0.206917
GBM_2_AutoML_20190718_142918,0.66671,0.667253,0.459336,0.210989


In [8]:
# The leader model is stored here
aml.leader

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_1_AutoML_20190718_142918


ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.2038320126018107
RMSE: 0.45147758815007716
LogLoss: 0.6581554069897447
Mean Per-Class Error: 0.6654259073483212
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,Error,Rate
1.0,566.0,1.0,0.9982394,567 / 568
0.0,7775.0,3.0,0.0003857,"3 / 7,778"
0.0,1700.0,4.0,0.9976526,"1,700 / 1,704"
1.0,10041.0,8.0,0.2258706,"2,270 / 10,050"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.7741293
2,0.9433831
3,1.0



ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.20891110581012015
RMSE: 0.4570679444132132
LogLoss: 0.6780942846759986
Mean Per-Class Error: 0.6663509977446386
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error
,2019-07-18 14:38:36,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan
,2019-07-18 14:38:37,18.532 sec,52220 obs/sec,1.0413469,1,20627.0,0.4543861,0.6688381,0.0320031,0.2268657
,2019-07-18 14:38:40,21.642 sec,60000 obs/sec,10.4079160,10,206160.0,0.4514776,0.6581554,0.0443558,0.2258706


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
mock_nw,1.0,1.0,0.0052996
mock_tw,0.9324063,0.9324063,0.0049414
charli_tw,0.9248908,0.9248908,0.0049016
charli_nw,0.8510520,0.8510520,0.0045103
yanke_tw,0.7962412,0.7962412,0.0042198
---,---,---,---
wh_tc,0.1902860,0.1902860,0.0010084
wh_tc,0.1891483,0.1891483,0.0010024
ash _tc,0.1721454,0.1721454,0.0009123
bir_tc,0.1682813,0.1682813,0.0008918



See the whole table with table.as_data_frame()


In [9]:
preds = aml.predict(test)
print(preds)

deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1,p2
1,0.0361144,0.839834,0.124051
1,0.045438,0.806089,0.148472
1,0.0540639,0.907661,0.0382749
1,0.0431976,0.813283,0.143519
1,0.0380133,0.818476,0.143511
1,0.0482067,0.787462,0.164331
1,0.05993,0.781062,0.159008
1,0.0453351,0.795927,0.158738
1,0.0591828,0.792589,0.148228
1,0.00931209,0.889024,0.101664


In [10]:
var = preds["predict"].cbind(test[y])
print(var)

predict,labels
1,1
1,2
1,1
1,1
1,1
1,1
1,2
1,1
1,1
1,2


In [11]:
# convert to pandas dataframe
y_test = h2o.as_list(test[y], use_pandas=True)
y_pred = h2o.as_list(preds["predict"])
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

[[   0  285    0]
 [   0 3835    1]
 [   0  830    1]]
0.7746365105008077
